In [165]:
import pandas as pd
import numpy as np
import requests
import json
import os
import time
import fred_msa

api_key = 'a37b50cd27afbc3ce23a81ddc5541dec'
endpoint = 'https://api.stlouisfed.org/fred/series/categories'

state_ids = fred_msa.get_state_ids(api_key)

msa_ids = pd.DataFrame()
for state, state_id in state_ids.items():
    state_dict = fred_msa.get_msa_cats(api_key, state_id)
    states = fred_msa.extract_state(pd.Series(state_dict.keys()))
    state_df = pd.DataFrame({'msa' : list(state_dict.keys()), 'state' : states, 'ID' : list(state_dict.values())})
    msa_ids = msa_ids.append(state_df)
    time.sleep(3)

msa_ids['city'] = fred_msa.split_city(msa_ids.msa)
msa_ids = msa_ids[msa_ids.msa.str.contains('CMSA') == False]
msa_ids = msa_ids.drop_duplicates(subset = ['city', 'state']).reset_index(drop = True)

# #msas_to_use = pd.read_csv('..\\fred-data\msas-to-use.csv', usecols=['city', 'state'])

# #msas_to_use = pd.merge(msas_to_use, msa_ids,
#          how='left', on=['city', 'state'], validate='one_to_one')

# #series = pd.DataFrame()

# for idx in range(len(msas_ids)):
#     msa_series = fred_msa.get_msa_series_list(msas_ids.ID[idx], api_key)
#     msa_series['msa'] = msas_ids.msa[idx]
#     msa_series['city'] = msas_ids.city[idx]
#     msa_series['state'] = msas_ids.state[idx]
    
#     series = series.append(msa_series).reset_index(drop=True)
#     time.sleep(3)
    

# series.to_csv('..\\fred-data\msa_series.csv', index=False)

<ipython-input-165-a6bb967f0ec0>:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  states = fred_msa.extract_state(pd.Series(state_dict.keys()))


In [ ]:
series = pd.DataFrame()

for idx in range(len(msa_ids)):
    msa_series = fred_msa.get_msa_series_list(msa_ids.ID[idx], api_key)
    msa_series['msa'] = msa_ids.msa[idx]
    msa_series['city'] = msa_ids.city[idx]
    msa_series['state'] = msa_ids.state[idx]
    
    series = series.append(msa_series).reset_index(drop=True)
    time.sleep(3)

In [166]:
msa_ids

,msa,state,ID,city
0,"Anniston-Oxford, AL",AL,30530.0,Anniston
1,"Auburn-Opelika, AL",AL,30531.0,Auburn
2,"Birmingham-Hoover, AL",AL,30532.0,Birmingham
3,"Columbus, GA-AL",GA,30533.0,Columbus
4,"Daphne-Fairhope-Foley, AL",AL,33417.0,Daphne
...,...,...,...,...
414,"Racine, WI",WI,30992.0,Racine
415,"Sheboygan, WI",WI,30993.0,Sheboygan
416,"Wausau-Weston, WI",WI,30994.0,Wausau
417,"Casper, WY",WY,30996.0,Casper


In [154]:
ny = series.query("city == 'New York'")
ny_housing = ny.title[ny.title.str.contains('Housing Inventory')]
housing_series = [s.split(' in')[0] for s in ny_housing]

In [158]:
housing_series

['Housing Inventory: Active Listing Count',
 'Housing Inventory: Active Listing Count Month-Over-Month',
 'Housing Inventory: Active Listing Count Year-Over-Year',
 'Housing Inventory: Average Listing Price',
 'Housing Inventory: Average Listing Price Month-Over-Month',
 'Housing Inventory: Average Listing Price Year-Over-Year',
 'Housing Inventory: Median Days on Market',
 'Housing Inventory: Median Days on Market Month-Over-Month',
 'Housing Inventory: Median Days on Market Year-Over-Year',
 'Housing Inventory: Median Listing Price',
 'Housing Inventory: Median Listing Price Month-Over-Month',
 'Housing Inventory: Median Listing Price per Square Feet',
 'Housing Inventory: Median Listing Price per Square Feet Month-Over-Month',
 'Housing Inventory: Median Listing Price per Square Feet Year-Over-Year',
 'Housing Inventory: Median Listing Price Year-Over-Year',
 'Housing Inventory: Median Home Size',
 'Housing Inventory: Median Home Size',
 'Housing Inventory: Median Home Size',
 'Hous

In [163]:
ny_employees = ny.title[ny.title.str.contains('All Employees')]
employees_series = list(np.unique([s.split(' in')[0] for s in ny_employees]))
employees_series

['All Employees: Accommodation and Food Services',
 'All Employees: Administrative and Support and Waste Management and Remediation Services',
 'All Employees: Education and Health Services',
 'All Employees: Education and Health Services: Health Care and Social Assistance',
 'All Employees: Federal Government',
 'All Employees: Financial Activities',
 'All Employees: Financial Activities: Finance and Insurance',
 'All Employees: Financial Activities: Insurance Carriers and Related Activities',
 'All Employees: Goods Producing',
 'All Employees: Government',
 'All Employees: Government: Federal Government',
 'All Employees: Government: Local Government',
 'All Employees: Government: State Government',
 'All Employees: Health Care and Social Assistance',
 'All Employees: Health Care: Ambulatory Health Care Services',
 'All Employees: Health Care: Hospitals',
 'All Employees: Information',
 'All Employees: Information: Telecommunications',
 'All Employees: Leisure and Hospitality',
 'All

In [164]:
state_ids

{'Alabama': 27282,
 'Alaska': 27283,
 'Arizona': 27284,
 'Arkansas': 149,
 'California': 27286,
 'Colorado': 27287,
 'Connecticut': 27288,
 'Delaware': 27289,
 'District of Columbia': 27290,
 'Florida': 27291,
 'Georgia': 27292,
 'Hawaii': 27293,
 'Idaho': 27294,
 'Illinois': 150,
 'Indiana': 151,
 'Iowa': 27297,
 'Kansas': 27298,
 'Kentucky': 152,
 'Louisiana': 27300,
 'Maine': 27301,
 'Maryland': 27302,
 'Massachusetts': 27303,
 'Michigan': 27304,
 'Minnesota': 27305,
 'Mississippi': 153,
 'Missouri': 154,
 'Montana': 27308,
 'Nebraska': 27309,
 'Nevada': 27310,
 'New Hampshire': 27311,
 'New Jersey': 27312,
 'New Mexico': 27313,
 'New York': 27314,
 'North Carolina': 27315,
 'North Dakota': 27316,
 'Ohio': 27317,
 'Oklahoma': 27318,
 'Oregon': 27319,
 'Pennsylvania': 27320,
 'Puerto Rico': 33504,
 'Rhode Island': 27322,
 'South Carolina': 27323,
 'South Dakota': 27324,
 'Tennessee': 193,
 'Texas': 27326,
 'Utah': 27328,
 'Vermont': 27329,
 'Virgin Islands': 33605,
 'Virginia': 27330